In [1]:
from core.agent import agent_executor, agent_llm
from core.CustomTools import fetch_and_rerank

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import (
    AgentTokenBufferMemory,
)


from datasets import Dataset

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

import regex as re
import glob

from core.ragas import RagasCallbackHandler

# ---

#agent_memory = AgentTokenBufferMemory(llm=agent_llm)
callback_handler = RagasCallbackHandler()
def query_agent(prompt):
   response = agent_executor(
            {"input": prompt, "history": []},                        
            callbacks=[callback_handler],
            include_run_info=True,
        )
   return response



In [18]:
# source data

filenames = []
for _file in glob.glob("ragas_data/*.txt"):
    filenames.append(_file)

questions = []
ground_truths = []

DELIM = "__###__"
for name in filenames:
    file_content = None
    with open(name) as f:                
        file_content = f.read()

    content = file_content.split(DELIM)
    questions.append(content[0])
    truths = []
    truths.append(content[1])
    ground_truths.append(truths)

print("Questions:", len(questions))
print("Grounded Truths:", len(ground_truths))
    

Questions: 2
Grounded Truths: 2


In [19]:
answers = []
contexts = []

# Inference
for query in questions:
  
  # retrieve an answer
  response = query_agent(query)
  answers.append(response["output"])

  inputs, outputs = callback_handler.clear_collected_outputs()
  print("Input: ", len(inputs))
  print("Outputs: ", len(outputs))

  results = []  
  [results.append(doc) for doc in outputs]          
  contexts.append(results)  

print("Answer len:", len(contexts))
print("Context len:", len(contexts))





> Entering new AgentExecutor chain...
starting tool ...  event aggregation in Apache Camel
Embedding ms:  0.33567118644714355
0.831 :  _faq_how-do-the-direct-event-seda-and-vm-endpoints-compare.html.txt
0.829 :  _others_cloudevents.html.txt
0.82 :  _eips_aggregate-eip.html.txt_10
0.829 :  _next_timer-component.html.txt_3
0.83 :  _eips_split-eip.html.txt_10
ending tool ... 

> Finished chain.
Input:  1
Outputs:  1


> Entering new AgentExecutor chain...
starting tool ...  file
Embedding ms:  0.3491039276123047
0.782 :  _next_file-component.html.txt_19
0.772 :  _next_sftp-component.html.txt_13
0.78 :  _eips_competing-consumers.html.txt_2
0.78 :  _next_ftp-component.html.txt_21
0.774 :  _next_file-component.html.txt_0
ending tool ... 
starting tool ...  AWS S3
ending tool ... 

> Finished chain.
Input:  2
Outputs:  2
Answer len: 2
Context len: 2


In [17]:
# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

df.head()

evaluating with [context_precision]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.95s/it]


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.60s/it]


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.36s/it]


evaluating with [answer_relevancy]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.47s/it]


,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,"I want to read a local file, create equal chun...","To read a local file, create equal chunks, and...","[page_content='""from(""file://inbox?preMove=inp...","[\nTo read a local file, create equal chunks, ...",0.0,1.0,0.555556,0.763375
